In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers torch numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

In [4]:
# 读取标签列表
label_path = '/content/drive/MyDrive/bert_test/obj.txt'
with open(label_path, 'r') as f:
    object_classes = [line.strip() for line in f.readlines()]

print(f"共读取到 {len(object_classes)} 个物体标签，示例：{object_classes[:5]}")
label_path1 = '/content/drive/MyDrive/bert_test/rel.txt'
with open(label_path1, 'r') as f:
    predicate_classes = [line.strip() for line in f.readlines()]

print(f"共读取到 {len(predicate_classes)} 个物体标签，示例：{predicate_classes[:5]}")

共读取到 100 个物体标签，示例：['person', 'sky', 'building', 'truck', 'bus']
共读取到 70 个物体标签，示例：['on', 'wear', 'has', 'next to', 'sleep next to']


In [5]:
import torch
from transformers import BertTokenizer, BertModel

# 选择模型（如果显存不足，可改用 "bert-base-uncased"）
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()  # 切换到评估模式

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
def get_bert_embedding(text, model, tokenizer):
    # 分词并编码
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=32  # 限制最大长度
    )

    # 提取特征
    with torch.no_grad():
        outputs = model(**inputs)

    # 平均池化所有 token 的特征
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

In [7]:
import numpy as np

# 存储所有标签的 BERT 向量（维度 768）
object_embeddings = []

for label in object_classes:
    embedding = get_bert_embedding(label, model, tokenizer)
    object_embeddings.append(embedding)

# 转换为 numpy 数组
object_embeddings = np.array(object_embeddings)
print("特征矩阵形状:", object_embeddings.shape)  # 应为 (N, 768)

特征矩阵形状: (100, 768)


In [8]:
save_path = '/content/drive/MyDrive/bert_test/vrd_objects_vec_bert.npy'
np.save(save_path, object_embeddings)
print(f"BERT 语义特征已保存至 {save_path}")

BERT 语义特征已保存至 /content/drive/MyDrive/bert_test/vrd_objects_vec_bert.npy


In [9]:
# 存储所有谓词标签的 BERT 向量（维度 768）
predicate_embeddings = []

for label in predicate_classes:
    embedding = get_bert_embedding(label, model, tokenizer)
    predicate_embeddings.append(embedding)

# 转换为 numpy 数组
predicate_embeddings = np.array(predicate_embeddings)
print("谓词特征矩阵形状:", predicate_embeddings.shape)  # 应为 (M, 768)

# 保存谓词向量
save_path_pred = '/content/drive/MyDrive/bert_test/vrd_predicates_vec_bert.npy'
np.save(save_path_pred, predicate_embeddings)
print(f"BERT 谓词语义特征已保存至 {save_path_pred}")

谓词特征矩阵形状: (70, 768)
BERT 谓词语义特征已保存至 /content/drive/MyDrive/bert_test/vrd_predicates_vec_bert.npy


#查看文件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np

# 指定文件路径
file_path = "/content/drive/MyDrive/vrd/vrd_roidb.npz"

# 加载 npz 文件
data = np.load(file_path, allow_pickle=True)

# 查看 npz 文件中的所有键
print("文件包含的键:", data.files)


文件包含的键: ['roidb']


In [8]:
import numpy as np

file_path = "/content/drive/MyDrive/vrd/vrd_roidb.npz"

# 加载时添加 encoding='latin1'
data = np.load(file_path, allow_pickle=True, encoding="latin1")

# 查看包含的键
print("文件包含的键:", data.files)

# 获取 roidb 数据
roidb = data["roidb"]

# 查看数据类型
print("roidb 类型:", type(roidb))

roidb = roidb.item()  # 取出存储的对象
print("解包后的 roidb 类型:", type(roidb))
print("roidb 的所有键:", roidb.keys())




文件包含的键: ['roidb']
roidb 类型: <class 'numpy.ndarray'>
解包后的 roidb 类型: <class 'dict'>
roidb 的所有键: dict_keys(['train_roidb', 'test_roidb'])


In [9]:
train_roidb = roidb["train_roidb"]
test_roidb = roidb["test_roidb"]

print("train_roidb 类型:", type(train_roidb))
print("test_roidb 类型:", type(test_roidb))

train_roidb 类型: <class 'list'>
test_roidb 类型: <class 'list'>


image: 可能是图像的路径或其他相关信息。

rela_gt: 关系的 ground truth（真实标注）。

height: 图像的高度。

width: 图像的宽度。

sub_gt: 主体（subject）的 ground truth。

index_pred: 预测的索引。

sub_box_gt: 主体框（ground truth）。

obj_gt: 客体（object）的 ground truth。

obj_box_gt: 客体框（ground truth）。



In [11]:
print("train_roidb 长度:", len(train_roidb))  # 训练集样本数量
print("test_roidb 长度:", len(test_roidb))
# 查看第一个样本的类型
first_sample = train_roidb[0]
print("第一个样本的类型:", type(first_sample))

# 如果是字典，打印它的键
if isinstance(first_sample, dict):
    print("第一个样本的键:", first_sample.keys())


train_roidb 长度: 3780
test_roidb 长度: 954
第一个样本的类型: <class 'dict'>
第一个样本的键: dict_keys(['image', 'rela_gt', 'height', 'width', 'sub_gt', 'index_pred', 'sub_box_gt', 'obj_gt', 'obj_box_gt'])
